# Computing expected values

In this notebook we use Python to compute expected values of random variables.
This is a very important "ingredient" in dynamic programming methods,
used to evaluate and find policies in small Markov Decision Process settings.

*<span style="color:red">Below, the parts indicated by `#??` need to be filled in!</span>*

### Example 1: normal dice

Consider a fair, 6-sided dice. We want to find the expected value of the number shown after a roll.
This can easily be computed to be $1/6 \sum_{k=1}^6 k = 3.5$.
In more complicated settings, it is often easier to iterate over possible outcomes and compute the expectation in a more naive fashion.

In [3]:
## Function that computes the expected payout
def ePayout():
    numbers = [1, 2, 3, 4, 5, 6] # ?? Possible numbers shown
    probs = [1/6]*6 # ?? Probability of each number
    E = 0
    for n, p in zip(numbers, probs):
        E += n * p # ?? Compute "contribution" of this outcome
    return E

print(ePayout())

3.5


### Example 2: $n$-sided dice

Consider the following "game":
The player rolls a fair $n$-sided dice (labelled $1, 2, \dots, n$) and receives a payout of $X^2$,
where $X$ denotes the number shown by the dice.
What is the expected payout?

In [3]:
## Function that computes the expected payout
def ePayout(N: int):
    numbers = list(range(1, N+1)) # ?? Possible numbers shown
    probs = [1/N]*N # ?? Probability of each number
    E = 0
    for n, p in zip(numbers, probs):
        E += n**2 * p # ?? Compute "contribution" of this outcome
    return E

# Test for some value of n
print(ePayout(7))

20.0


*Bonus: this value can also be derived algebraically, using the identity $\sum_{x=1}^n x^2 = n(n+1)(2n+1)/6$.*

### Example 3: two-step game

Consider the following "game":
The player rolls a fair $n$-sided dice,
let $X$ denote the result of this roll.
If $X$ is odd, the player receives a payout of $-1$.
If $X$ is even, the player draws a card from a deck labelled $(-2, -1, 0, 1, \dots, X/2)$
and receives a payout equal to the number drawn.
What is the expected payout?

**ANSWER**

For an even result $X$, let us define $x = X/2$. The expectation associated with outcome $X$ is
$$ \dfrac{1}{3 + x} \left((-2) + (-1) + 0 + 1 + \ldots + x \right) $$
It can be rewritten as
$$ \dfrac{1}{3 + x} \left(\dfrac{1}{2} x (x+1) -3\right) = \dfrac{x-2}{2}$$

The computation of the expectation depends on if the number of sides on the dice, $N$, is even or odd.
$$
\mathbb{E}[game] =
\begin{cases}
- \dfrac{1}{2} + \dfrac{1}{N}\sum\limits_{x=1}^{N/2} \dfrac{x-2}{2} & \text{if $N$ is even} \\
- \dfrac{N+1}{2N} + \dfrac{1}{N}\sum\limits_{x=1}^{(N-1)/2} \dfrac{x-2}{2} & \text{if $N$ is odd}
\end{cases}
$$

This can be simplified into:

$$
\mathbb{E}[game] =
\begin{cases}
\dfrac{1}{16}(N -14) & \text{if $N$ is even} \\
\dfrac{1}{16N}(N^2 -16N -1)  & \text{if $N$ is odd}
\end{cases}
$$

In [52]:
## Function that computes the expected payout
def ePayout(N: int):
    if N % 2 == 0:
        return (N-14) / 16
    else:
        return (N**2 -16*N -1) / (16*N)

# Check values for some n
for x in range(1, 26):
    print(x, f'{ePayout(x):.4f}')

1 -1.0000
2 -0.7500
3 -0.8333
4 -0.6250
5 -0.7000
6 -0.5000
7 -0.5714
8 -0.3750
9 -0.4444
10 -0.2500
11 -0.3182
12 -0.1250
13 -0.1923
14 0.0000
15 -0.0667
16 0.1250
17 0.0588
18 0.2500
19 0.1842
20 0.3750
21 0.3095
22 0.5000
23 0.4348
24 0.6250
25 0.5600


### Monte Carlo

The expectations above can also be estimated using Monte-Carlo estimation,
repeating the experiment $N$ times for some large $N$.

In [5]:
# We need to generate random numbers to do this
import random

In [43]:
## Example 1
def mcPayout(N):
    xList = [0] * N
    for i in range(N):
        xList[i] = random.randint(1, 6)
    return sum(xList) / N

# Run MC
mcPayout(10000)

3.5237

In [7]:
## Example 2
def mcPayout(N, n):
    xList = [0] * N
    for i in range(N):
        xList[i] = random.randint(1, n)**2
    return sum(xList) / N

# Run MC
mcPayout(10000, 7)

20.0474

In [60]:
import numpy as np

def mcPayout(N, n):
    
    xList = np.random.choice(range(1, n+1), N, replace=True)
    xListOdd = [-1 for x in xList if x % 2 == 1]
    xListEven = [random.randint(-2, x/2) for x in xList if x % 2 == 0]

    return (sum(xListOdd) + sum(xListEven)) / N


# Run MC
for x in range(1, 6):
    print(mcPayout(100000, x))

-1.0
-0.74921
-0.83524
-0.62504
-0.70043
